In [23]:
#code to set working directory in google colabs
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("drive/My Drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [24]:
!pip install pomegranate
from pomegranate import *

In [0]:
import json
json_files = ['nf_10']
json_list = []

for i in json_files:
  filepath = "./Colab Notebooks/evello/data/SeaShips/AIS/"+i
  with open(filepath, 'r') as json_file:
    data = json.load(json_file)
    json_list.append(data)

In [26]:
print(json_list[0])

{'HEADING': 21.0, 'LAT': -75.0971, 'LON': 35.9868, 'MMSI': '366983550', 'MOVE_TYPE': '0', 'PORT_ID': '136', 'PORT_NAME': 'NORFOLK', 'SHIPNAME': 'OCEAN LADY', 'SHIP_ID': '3961225', 'SPEED': 12.32, 'TIMESTAMP_LT': '2020-04-13T05:45:00', 'TIMESTAMP_UTC': '2020-04-13T09:45:00', 'TYPE_NAME': 'FISHING_BOAT', 'UNLOCODE': 'USORF'}


In [0]:
def generateEvent(ais, image=None, text=None):
  #read in jsons and get characteristics
  if ais['PORT_NAME'] == 'SAN DIEGO':

    if -120.3442 <= ais['LAT'] <= -117.6526 and 32.3707 <= ais['LON'] <= 33.2933 :
      position = 'A'
    else:
      position = 'N'
    
    if 0 <= ais['SPEED'] <= 0.8 :
      speed = 'A'
    else:
      speed = 'N'

    if 290 <= ais['HEADING'] <= 350 :
      heading = 'A'
    else:
      heading = 'N'

  elif ais['PORT_NAME'] == 'NORFOLK':

    if -75.7837 <= ais['LAT'] <= -75.0256 and 35.7457 <= ais['LON'] <= 36.7917 :
      position = 'A'
    else:
      position = 'N'
    
    if 0 <= ais['SPEED'] <= 0.8 :
      speed = 'A'
    else:
      speed = 'N'

    if 300 <= ais['HEADING'] <= 340 :
      heading = 'A'
    else:
      heading = 'N'
  
  if image:
    test = 1
    #if count of images in ship overlap significantly:
      #boat_overlap = 'A'
    #else:
      #boat_overlap = 'N'
  else:
    boat_overlap = 'N'

  return heading, speed, position, boat_overlap

In [0]:
def bayes_network(heading, speed, position, boat_overlap):
  heading_dis = DiscreteDistribution({'A': 1./2, 'N': 1./2})
  speed_dis = DiscreteDistribution({'A': 1./2, 'N': 1./2})
  position_dis = DiscreteDistribution({'A': 1./2, 'N': 1./2})
  boat_overlap_dis = DiscreteDistribution({'A': 1./2, 'N': 1./2})

  event_dis = ConditionalProbabilityTable(
          [['A', 'A', 'A', 'A', 'A', 0.5],
          
          ['A', 'A', 'A', 'A', 'N', 0.2],
          ['A', 'A', 'A', 'N', 'A', 0.2],
          ['A', 'A', 'N', 'A', 'A', 0.6],
          ['A', 'N', 'A', 'A', 'A', 0.7],
          ['N', 'A', 'A', 'A', 'A', 0.0],

          ['N', 'N', 'A', 'A', 'A', 0.9],
          ['N', 'A', 'N', 'A', 'A', 0.4],
          ['N', 'A', 'A', 'N', 'A', 1.0],
          ['N', 'A', 'A', 'A', 'N', 1.0],
          ['A', 'N', 'N', 'A', 'A', 0.4],
          ['A', 'N', 'A', 'N', 'A', 0.3],
          ['A', 'N', 'A', 'A', 'N', 0.0],
          ['A', 'A', 'N', 'N', 'A', 0.0],
          ['A', 'A', 'N', 'A', 'N', 0.2],
          ['A', 'A', 'A', 'N', 'N', 0.2],

          ['A', 'A', 'N', 'N', 'N', 0.1],
          ['A', 'N', 'A', 'N', 'N', 0.2],
          ['A', 'N', 'N', 'A', 'N', 0.3],
          ['A', 'N', 'N', 'N', 'A', 0.4],
          ['N', 'A', 'A', 'N', 'N', 0.5],
          ['N', 'A', 'N', 'A', 'N', 0.6],
          ['N', 'A', 'N', 'N', 'A', 0.7],
          ['N', 'N', 'A', 'A', 'N', 0.8],
          ['N', 'N', 'A', 'N', 'A', 0.9],
          ['N', 'N', 'N', 'A', 'A', 1.0],

          ['A', 'N', 'N', 'N', 'N', 0.0],
          ['N', 'A', 'N', 'N', 'N', 0.0],
          ['N', 'N', 'A', 'N', 'N', 0.0],
          ['N', 'N', 'N', 'A', 'N', 0.0],
          ['N', 'N', 'N', 'N', 'A', 0.0],

          ['N', 'N', 'N', 'N', 'N', 0.0]], [heading_dis, speed_dis, position_dis, boat_overlap_dis])

  s1 = Node(heading_dis, name="heading")
  s2 = Node(speed_dis, name="speed")
  s3 = Node(position_dis, name="position")
  s4 = Node(boat_overlap_dis, name="boat_overlap")
  s5 = Node(event_dis, name="event")

  model = BayesianNetwork("ship_event_reasoning")
  model.add_states(s1, s2, s3, s4, s5)
  model.add_edge(s1, s5)
  model.add_edge(s2, s5)
  model.add_edge(s3, s5)
  model.add_edge(s4, s5)
  model.bake()
  pred = model.predict([[heading, speed, position, boat_overlap, None]])
  prob = model.probability(pred)
  return pred, prob


In [69]:
heading, speed, position, boat_overlap = generateEvent(json_list[0])
print(position, speed, heading, boat_overlap)
pred, prob, prob2 = bayes_network(position, speed, heading, boat_overlap)
print(pred,prob, prob2)


A N N N
[array(['A', 'N', 'N', 'N', 'A'], dtype=object)] 0.025 0.0


In [0]:
def event_reasoning(ais, image=None, text=None):  
  position, speed, heading, boat_overlap = generateEvent(ais)
  pred, prob = bayes_network(position, speed, heading, boat_overlap)
  return {'prediction':pred[0][4], 'probability':prob}

In [80]:
dict1 = event_reasoning(json_list[0])
print(dict1)

{'prediction': 'A', 'probability': 0.05625}
